In [2]:
!pip install selenium
!pip install webdriver-manager

In [2]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from tqdm import tqdm

In [3]:
df = pd.read_excel("1970_2020_raw영화데이터_kobiz_update.xlsx")
#df = df[['movieCd','movieNm', 'prdtYear']]
#df['stats'] = None
names = df.movieNm.astype(str)

df.head()

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,prdtStatNm,genres,directors,actors,showTypes,audits,Domestic,Budget,Distributor,review,MPAA,stats,raters,ratings
0,20203303,지니어스 독,Think Like a Dog,90.0,2020.0,20200916,개봉예정,"코미디,드라마",길 정거,"메간 폭스,조쉬 더하멜,가브리엘 베이트먼",2D,12세이상관람가,-,NaN,NaN,NaN,12,NaN,667,5.1
1,20175405,뉴 뮤턴트,The New Mutants,98.0,2018.0,20200903,개봉예정,"액션,공포(호러),SF",조쉬 분,"메이지 윌리암스,안야 테일러 조이,앨리스 브라가",2D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20112692,드라이브,Drive,100.0,2011.0,20111117,개봉,"액션,멜로/로맨스",니콜라스 윈딩 레픈,"라이언 고슬링,캐리 멀리건","필름,2D","청소년관람불가,청소년관람불가","$35,061,555","$15,000,000",FilmDistrict,NaN,NaN,144975500.0,NaN,NaN
3,20205511,쥬라기 썬더,Jurassic Thunder,84.0,2019.0,20200923,개봉예정,"액션,SF",NaN,NaN,2D,15세이상관람가,NaN,NaN,NaN,"....but then again, did you expect an award wi...",15,NaN,142,1.9
4,20203043,바다 몬스터2,Sea Monsters2,87.0,2018.0,20200812,개봉,애니메이션,에반 트라멜,"서반석,석승훈","2D,2D",전체관람가,NaN,NaN,NaN,Loose plot with no content. The story is vauge...,NaN,5000.0,30,3.7


In [52]:
df2 = df[df.Domestic.notnull()][df.Domestic !="-"]
indexes = df2.index.values
len(indexes)

3702

In [38]:
df['stats2'] = 0

In [39]:
addr = "http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do" 
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(addr)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\beomj\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache
 


SessionNotCreatedException: Message: session not created
from chrome not reachable
  (Session info: chrome=84.0.4147.135)


In [63]:
def find(name, cd):
    driver.find_element_by_name("sMovName").clear()
    driver.find_element_by_name("sMovName").send_keys(name)
    driver.find_element_by_class_name("btn_sch").click()
    time.sleep(1)
    try:
        found =False
        for i in range(2, 11):
            temp = driver.find_element_by_class_name("rst_sch")
            temp = temp.find_elements_by_link_text(name)
            for t in temp:
                t.click()
                temp = driver.find_element_by_class_name("item_tab")
                text = temp.find_element_by_class_name("cont").text
                if cd not in text:
                    driver.find_element_by_link_text("뒤로").click()   
                else:
                    found = True
                    break
            if found:
                break
            temp = driver.find_element_by_id("pagingForm")
            temp.find_element_by_link_text(str(i)).click()
            time.sleep(0.5)
        if not found:
            return None
    except:
        return None

    temp = driver.find_element_by_class_name("wrap_tab")
    temp.find_element_by_link_text('통계정보').click()
    while True:
        try:
            time.sleep(2) 
            temp = driver.find_element_by_class_name("statistics")
            infos = temp.find_elements_by_class_name("info")
            if infos:
                break
        except:
            pass
    
    total = []
    #found = False
    for j in infos:
        if "KOBIS(발권)통계" in j.text:
            tables = j.find_elements_by_tag_name("table")
            for i in tables:
                if '전국' in i.text:
                    #found=True
                    total.append(i.text[i.text.find("전국"):])
            break
            #if found:
        #    break
    driver.find_element_by_link_text("뒤로").click()   
    #if not found:
    #    return None
    #total = " ".join(total).split('\n')
    total = " ".join(total)
    #for i in total:
    return total

In [72]:
start = 61
end = 70
for i in tqdm(range(start, end)):
    if i in indexes:
   # if not df.loc[i, 'stats']:
        name = df.loc[i, 'movieNm'].strip()
        cd   = df.loc[i, 'movieCd']
        try:
            text = find(name, cd)
            df.loc[i, 'stats2'] = text
            print(i, text[:20])
        except:
            addr = "http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do" 
            driver.get(addr)
            df.loc[i, 'stats2'] = None

100%|██████████| 9/9 [00:50<00:00,  5.56s/it]69 전국 156 264,874,260 (



In [43]:
df.to_excel("1970_2019_코비스매출_stats2"+str(start)+"_"+str(end)+".xlsx", index=False)

In [73]:
pd.set_option('display.max_rows', 80)
df.loc[indexes].loc[start:end][['movieNm','stats', 'stats2']]

,movieNm,stats,stats2
61,배트맨 비긴즈,NaN,"전국 301 5,799,168,000 (100%) 903,190 (100%)"
62,라스트 풀 메저,88807180.0,"전국 333 88,807,180 (100%) 11,223 (100%)"
64,"색, 계",0.0,"전국 258 12,661,780,400 (100%) 1,925,774 (100%)"
65,잃어버린 세계를 찾아서,161271360.0,"전국 279 165,941,860 (100%) 21,879 (100%)"
68,리플리,0.0,"전국 19 20,213,500 (100%) 2,605 (100%)"
69,아이 캔 온리 이매진,182789600.0,"전국 156 264,874,260 (100%) 34,859 (100%)"
70,플라이,0.0,0
